# Request and download data for a list of instruments:
This is a step by step python code to build netCDF data request urls for pre-selected instruments in a file and send requests if prompted

### Input variables:

In [39]:
sDir = '/Users/leila/Documents/Desktop/tobe'#directory where outputs are saved
username = '' #OOI API username
token = '' # OOI API password
f = '/Users/leila/Documents/NSFEduSupport/github/8thEGOMeeting-notebooks/instrument_list.csv'# file with pre-selected list of instruments
# the following column names should in the file: ['array','subsite','node','sensor','delivery_method','reference_designator']


### Import functions:

In [40]:
import datetime as dt
import pandas as pd
import os
import itertools
import functions.common as cf
from scripts import data_request_urls_ooi1_0
from scripts import send_data_requests_nc
from scripts import data_request_tools

### Read input file:

In [41]:
df = pd.read_csv(os.path.join(sDir, f))

### Create request urls for instruments in the read file:

In [44]:
now = dt.datetime.now().strftime('%Y%m%dT%H%M')
url_list = []
for i, j in df.iterrows():
    array = data_request_tools.check_str(j['array'])
    array = data_request_tools.format_inputs(array)
    refdes = j['reference_designator']
    if type(refdes) == str:
        subsite = data_request_tools.format_inputs(refdes.split('-')[0])
        node = data_request_tools.format_inputs(refdes.split('-')[1])
        inst = data_request_tools.format_inputs('-'.join((refdes.split('-')[2], refdes.split('-')[3])))
    else:
        subsite = data_request_tools.check_str(j['subsite'])
        subsite = data_request_tools.format_inputs(subsite)
        node = data_request_tools.check_str(j['node'])
        node = data_request_tools.format_inputs(node)
        inst = data_request_tools.check_str(j['sensor'])
        inst = data_request_tools.format_inputs(inst)
    delivery_methods = data_request_tools.check_str(j['delivery_method'])
    delivery_methods = data_request_tools.format_inputs(delivery_methods)
    
    print(refdes, delivery_methods)
    urls = data_request_urls_ooi1_0.main(sDir, array, subsite, node, inst, delivery_methods, now)
    url_list.append(urls)
    print(url_list)

CP05MOAS-GL336-03-CTDGVM000 ['telemetered']
[['https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS/GL336/03-CTDGVM000/telemetered/ctdgv_m_glider_instrument?beginDT=2015-01-11T20:57:00.000Z&endDT=2018-04-03T11:50:00.000Z&include_annotations=true&include_provenance=true']]
CP05MOAS-GL336-04-DOSTAM000 ['telemetered']
[['https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS/GL336/03-CTDGVM000/telemetered/ctdgv_m_glider_instrument?beginDT=2015-01-11T20:57:00.000Z&endDT=2018-04-03T11:50:00.000Z&include_annotations=true&include_provenance=true'], ['https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS/GL336/04-DOSTAM000/telemetered/dosta_abcdjm_glider_instrument?beginDT=2015-01-11T20:57:00.000Z&endDT=2018-04-03T11:50:00.000Z&include_annotations=true&include_provenance=true']]
CP05MOAS-GL336-05-PARADM000 ['telemetered']
[['https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS/GL336/03-CTDGVM000/telemetered/ctdgv_m_glider

### Send url requests:

In [45]:
f_url_list = list(itertools.chain(*url_list))

thredds_output_urls = send_data_requests_nc.main(sDir, f_url_list, username, token, now)


There are 8 requests to send, are you sure you want to continue? y/<n>: y

Request url 1 of 8: https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS/GL336/03-CTDGVM000/telemetered/ctdgv_m_glider_instrument?beginDT=2015-01-11T20:57:00.000Z&endDT=2018-04-03T11:50:00.000Z&include_annotations=true&include_provenance=true
Data request sent
Data available for request

Request url 2 of 8: https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS/GL336/04-DOSTAM000/telemetered/dosta_abcdjm_glider_instrument?beginDT=2015-01-11T20:57:00.000Z&endDT=2018-04-03T11:50:00.000Z&include_annotations=true&include_provenance=true
Data request sent
Data available for request

Request url 3 of 8: https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS/GL336/05-PARADM000/telemetered/parad_m_glider_instrument?beginDT=2015-01-11T20:57:00.000Z&endDT=2018-04-03T11:50:00.000Z&include_annotations=true&include_provenance=true
Data request sent
Data available for requ

### Seeing if the requests have fulfilled:

In [46]:
for i in range(len(thredds_output_urls)):
    url = thredds_output_urls[i]
    print('\nDataset {} of {}: {}'.format((i + 1), len(thredds_output_urls), url))
    if 'no_output_url' not in url:
        cf.check_request_status(url)


Dataset 1 of 8: https://opendap.oceanobservatories.org/thredds/catalog/ooi/leila.ocean@gmail.com/20190322T153940-CP05MOAS-GL336-03-CTDGVM000-telemetered-ctdgv_m_glider_instrument/catalog.html
Data request is still fulfilling. Trying again in 1 minute.
Data request is still fulfilling. Trying again in 1 minute.
Data request is still fulfilling. Trying again in 1 minute.
Data request has fulfilled.

Dataset 2 of 8: https://opendap.oceanobservatories.org/thredds/catalog/ooi/leila.ocean@gmail.com/20190322T153941-CP05MOAS-GL336-04-DOSTAM000-telemetered-dosta_abcdjm_glider_instrument/catalog.html
Data request has fulfilled.

Dataset 3 of 8: https://opendap.oceanobservatories.org/thredds/catalog/ooi/leila.ocean@gmail.com/20190322T153941-CP05MOAS-GL336-05-PARADM000-telemetered-parad_m_glider_instrument/catalog.html
Data request has fulfilled.

Dataset 4 of 8: https://opendap.oceanobservatories.org/thredds/catalog/ooi/leila.ocean@gmail.com/20190322T153941-CP05MOAS-GL336-02-FLORTM000-telemetere